In [1]:
!pip install git+https://github.com/huggingface/transformers accelerate flash_attn
!pip install qwen_vl_utils av

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-6u2b08j8
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-6u2b08j8
  Resolved https://github.com/huggingface/transformers to commit deac971c469bcbb182c2e52da0b82fb3bf54cccf
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 26.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 43.2 MB/s eta 0:00:00
  Created wheel for transformers: filename=transformers-4.48.0.dev0-py3-none-any.whl size=10236313 sha256=b6a8be26511691f98d0b7b740b89fbde123527760a2ed238137dbb7cd1a8ecea
  Stored in directory: /tmp/pip-ephem-wheel-cache-cjneqfit/wheels/c0/14/d6/6c9a5582d2ac191ec0a483be151a4495fe1eb2a6706ca49f1b
  Created wheel for flash_attn: filename=flash_attn-2

In [2]:
from transformers import Qwen2VLForConditionalGeneration, AutoTokenizer, AutoProcessor
from qwen_vl_utils import process_vision_info
import torch

In [3]:
model_name = "Qwen/Qwen2-VL-2B-Instruct"

In [14]:
model = Qwen2VLForConditionalGeneration.from_pretrained(
model_name, torch_dtype=torch.bfloat16, attn_implementation="eager", device_map="auto"
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [15]:
processor = AutoProcessor.from_pretrained(model_name)

In [26]:

messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "video",
                "video": "/content/3197638-hd_1920_1080_25fps.mp4",
                "max_pixels": 360 * 420,
                "fps": 1.0,
            },
            {"type": "text", "text": "HOw many doctors are there and what they are doing?."},
        ],
    }
]

In [27]:
messages

[{'role': 'user',
  'content': [{'type': 'video',
    'video': '/content/3197638-hd_1920_1080_25fps.mp4',
    'max_pixels': 151200,
    'fps': 1.0},
   {'type': 'text',
    'text': 'HOw many doctors are there and what they are doing?.'}]}]

In [28]:
text = processor.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)

In [29]:
image_inputs, video_inputs = process_vision_info(messages)
inputs = processor(
    text=[text],
    images=image_inputs,
    videos=video_inputs,
    padding=True,
    return_tensors="pt",
)

In [30]:
inputs = inputs.to("cuda")

In [31]:
generated_ids = model.generate(**inputs, max_new_tokens=512)

In [32]:
generated_ids

tensor([[151644,   8948,    198,   2610,    525,    264,  10950,  17847,     13,
         151645,    198, 151644,    872,    198, 151652, 151656, 151656, 151656,
         151656, 151656, 151656, 151656, 151656, 151656, 151656, 151656, 151656,
         151656, 151656, 151656, 151656, 151656, 151656, 151656, 151656, 151656,
         151656, 151656, 151656, 151656, 151656, 151656, 151656, 151656, 151656,
         151656, 151656, 151656, 151656, 151656, 151656, 151656, 151656, 151656,
         151656, 151656, 151656, 151656, 151656, 151656, 151656, 151656, 151656,
         151656, 151656, 151656, 151656, 151656, 151656, 151656, 151656, 151656,
         151656, 151656, 151656, 151656, 151656, 151656, 151656, 151656, 151656,
         151656, 151656, 151656, 151656, 151656, 151656, 151656, 151656, 151656,
         151656, 151656, 151656, 151656, 151656, 151656, 151656, 151656, 151656,
         151656, 151656, 151656, 151656, 151656, 151656, 151656, 151656, 151656,
         151656, 151656, 151

In [33]:
generated_ids_trimmed = [
    out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
]

In [34]:
output_text = processor.batch_decode(
    generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
)


In [35]:
print(output_text)

['There are four doctors in the image. They are standing in a hospital room, possibly preparing for or observing a medical procedure.']
